In [1]:
import clusters_builder
import visualization

In [2]:
cluster_builder=clusters_builder.ClustersBuilderCountries()

In [15]:
dates=['15.04.20']

In [8]:
map_builder=visualization.MapBuilderCountries()

In [16]:
for date in dates:
    print('\n', date, end=' ')
    map_builder.save_map(cluster_builder, date)


 15.04.20 n_clust:  15 [1, 2, 77, 2, 4, 2, 34, 1, 45, 7, 2, 1, 6, 2, 1] 

In [17]:
map_builder.save_as_img(dates)

15.04.20
